In [1]:
from queryportal.subgraphinterface import SubgraphInterface

from datetime import datetime, timedelta

import polars as pl
pl.Config.set_fmt_str_lengths(200)

polars.config.Config

To query from a decentralized subgraph, sign up for a playgrounds API key [here](https://app.playgrounds.network/login). 
Then make an `.env` file in the root directory with the following contents:

```
PLAYGROUNDS_API_KEY=<your api key>
```

For more information on querying from a decentralized subgraph, see the [documentation](https://docs.playgrounds.network/gateway/querying/)


In [2]:
# instantiate Dex class with subgraph key
univ2 = '2szAn45skWZFLPUbxFEtjiEzT1FMW8Ff5ReUPbZbQxtt'
univ3 = 'G3JZhmKKHC4mydRzD6kSz5fCWve5WDYYCyTFSJyv3SD5' 

# NOTE - Decentralized endpoints do not have a nice naming convention. Thus a dictionary is required to explicitly name the endpoints when they are instantiated.
sgi = SubgraphInterface(
    endpoints={'univ2': f'https://api.playgrounds.network/v1/proxy/subgraphs/id/{univ2}',       # https://thegraph.com/explorer/subgraphs/2szAn45skWZFLPUbxFEtjiEzT1FMW8Ff5ReUPbZbQxtt?view=Overview?chain=mainnet
                'univ3': f'https://api.playgrounds.network/v1/proxy/subgraphs/id/{univ3}'       # https://thegraph.com/explorer/subgraphs/G3JZhmKKHC4mydRzD6kSz5fCWve5WDYYCyTFSJyv3SD5?view=Overview?chain=mainnet
               }
    )

print(f'subject endpoints: {sgi.subject.subgraphs.keys()}')

subject endpoints: dict_keys(['univ2', 'univ3'])


In [3]:
start_time = int((datetime(2023, 4, 12).timestamp()))
end_time = int(datetime(2023, 4, 19).timestamp())
query_size = 2500

In [4]:
univ3_filter = {
    'pool_name_contains': 'Pepe',
    'timestamp_gte': start_time,
    'timestamp_lte': end_time
}

univ3_cols = [
    'timestamp',
    'hash',
    # comes from Pool! entity
    'pool_name', 
    'pool_id',
    # comes from Token entity
    'tokenIn_symbol', 
    'tokenOut_symbol', 
    'tokenIn_id',
    'tokenOut_id',
    ]

univ3_df = sgi.query_entity(
    entity='swaps',
    name='univ3',
    query_paths=univ3_cols,
    filter_dict = univ3_filter,
    query_size=query_size
    )

Querying endpoint: univ3
 Queryable schema entities: dict_keys(['tokens', 'rewardTokens', 'liquidityPoolFees', 'dexAmmProtocols', 'usageMetricsDailySnapshots', 'usageMetricsHourlySnapshots', 'financialsDailySnapshots', 'liquidityPools', 'liquidityPoolDailySnapshots', 'liquidityPoolHourlySnapshots', 'ticks', 'tickDailySnapshots', 'tickHourlySnapshots', 'accounts', 'positions', 'positionSnapshots', 'deposits', 'withdraws', 'swaps', 'activeAccounts', 'liquidityPoolAmounts', 'helperStores', 'tokenWhitelists', 'tokenWhitelistSymbols', 'protocols'])
Shape: (2500, 8)
Schema: {'timestamp': Int64, 'hash': Utf8, 'pool_name': Utf8, 'pool_id': Utf8, 'tokenIn_symbol': Utf8, 'tokenIn_id': Utf8, 'tokenOut_symbol': Utf8, 'tokenOut_id': Utf8}
Function 'query_entity' executed in 7.0928s


In [5]:
# We know that the PEPE address is `0x6982508145454ce325ddbe47a25d4ec3d2311933`...
univ2_filter = {
    'pair_id': '0xa43fe16908251ee70ef74718545e4fe6c5ccec9f',
    'timestamp_gte': start_time,
    'timestamp_lte': end_time
}

univ2_cols = [
    'timestamp',
    'sender',
    'amountUSD',
    'transaction_id',
    'pair_id',
    'sender',
    'to',
    'from',
    'pair_token0_id',
    'pair_token1_id',
    'amount0In',
    'amount0Out',
    'amount1In',
    'amount1Out'
    ]

univ2_df = sgi.query_entity(
    entity='swaps',
    name='univ2',
    query_paths=univ2_cols,
    filter_dict = univ2_filter,
    query_size=query_size
    )

Querying endpoint: univ2
 Queryable schema entities: dict_keys(['uniswapFactories', 'tokens', 'pairs', 'users', 'liquidityPositions', 'liquidityPositionSnapshots', 'transactions', 'mints', 'burns', 'swaps', 'bundles', 'uniswapDayDatas', 'pairHourDatas', 'pairDayDatas', 'tokenDayDatas'])
Shape: (2500, 13)
Schema: {'timestamp': Int64, 'amountUSD': Float64, 'sender': Utf8, 'to': Utf8, 'from': Utf8, 'amount0In': Float64, 'amount0Out': Float64, 'amount1In': Float64, 'amount1Out': Float64, 'transaction_id': Utf8, 'pair_id': Utf8, 'pair_token0': Struct([Field('id', Utf8)]), 'pair_token1': Struct([Field('id', Utf8)])}
Function 'query_entity' executed in 8.2968s


In [6]:
univ2_df.head(5)

timestamp,amountUSD,sender,to,from,amount0In,amount0Out,amount1In,amount1Out,transaction_id,pair_id,pair_token0,pair_token1
i64,f64,str,str,str,f64,f64,f64,f64,str,str,struct[1],struct[1]
1681518551,2155.606202,"""0x555b6ee8fab3dfdbcca9121721c435fd4c7a1fd1""","""0x555b6ee8fab3dfdbcca9121721c435fd4c7a1fd1""","""0x0e63bbdc6984101349fde52ee99bb06a160442ed""",0.0,2.6374e12,0.997,0.0,"""0x00032d7e3a9ecf6135d46b5f158534e652bda2113b76a611f49ad877406538ca""","""0xa43fe16908251ee70ef74718545e4fe6c5ccec9f""","{""0x6982508145454ce325ddbe47a25d4ec3d2311933""}","{""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""}"
1681834451,15204.44035,"""0x6b75d8af000000e20b7a7ddf000ba900b4009a80""","""0x6b75d8af000000e20b7a7ddf000ba900b4009a80""","""0xae2fc483527b8ef99eb5d9b44875f005ba1fae13""",0.0,1.2256e11,7.25511,0.0,"""0x0004bcb4d7950add5bb14aff891985287b21c099b53e3e294496ef2cedbd7492""","""0xa43fe16908251ee70ef74718545e4fe6c5ccec9f""","{""0x6982508145454ce325ddbe47a25d4ec3d2311933""}","{""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""}"
1681715519,1556.849838,"""0x6b75d8af000000e20b7a7ddf000ba900b4009a80""","""0x6b75d8af000000e20b7a7ddf000ba900b4009a80""","""0xae2fc483527b8ef99eb5d9b44875f005ba1fae13""",0.0,2.2888e10,0.741382,0.0,"""0x0005755e29bdcf849353a3dc9b7fc8727b19dfe625b7285d6c24451131e0ac08""","""0xa43fe16908251ee70ef74718545e4fe6c5ccec9f""","{""0x6982508145454ce325ddbe47a25d4ec3d2311933""}","{""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""}"
1681749647,3311.609379,"""0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b""","""0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b""","""0x5eb104b3aa1a43843f1be4e4205e2720834ed938""",1.1469e11,0.0,0.0,1.591434,"""0x00069db917db562f0625f4650e9f6791758a5864ab336f52b2a483dac84380e9""","""0xa43fe16908251ee70ef74718545e4fe6c5ccec9f""","{""0x6982508145454ce325ddbe47a25d4ec3d2311933""}","{""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""}"
1681710539,50.274263,"""0x1111111254eeb25477b68fb85ed929f73a960582""","""0x1111111254eeb25477b68fb85ed929f73a960582""","""0x27be222515fcbeff40781573a1ea2d630d0956de""",1.0000e9,0.0,0.0,0.023952,"""0x0009e67a6a0c3dd5bb5004488e6ec80a5e88c67854bf120049184c05a4c18c2a""","""0xa43fe16908251ee70ef74718545e4fe6c5ccec9f""","{""0x6982508145454ce325ddbe47a25d4ec3d2311933""}","{""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""}"


In [7]:
univ3_df.head(5)

timestamp,hash,pool_name,pool_id,tokenIn_symbol,tokenIn_id,tokenOut_symbol,tokenOut_id
i64,str,str,str,str,str,str,str
1681853891,"""0x00088858686149d26d6a1ed13c994bac7ece080e90925495213a8349523a469c""","""Uniswap V3 Pepe/Wrapped Ether 1%""","""0xf239009a101b6b930a527deaab6961b6e7dec8a6""","""WETH""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""PEPE""","""0x6982508145454ce325ddbe47a25d4ec3d2311933"""
1681748555,"""0x000cd2f5d66d13d43a934ee4a5b7d9911c680eb0404a689c44e0b2583e55ee28""","""Uniswap V3 Pepe/Wrapped Ether 1%""","""0xf239009a101b6b930a527deaab6961b6e7dec8a6""","""WETH""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""PEPE""","""0x6982508145454ce325ddbe47a25d4ec3d2311933"""
1681730903,"""0x000dd4330a3fb6af76221d43e21571b93e409c5c40e11f8e3f3624fa4a2672e5""","""Uniswap V3 Pepe/Wrapped Ether 1%""","""0xf239009a101b6b930a527deaab6961b6e7dec8a6""","""WETH""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""PEPE""","""0x6982508145454ce325ddbe47a25d4ec3d2311933"""
1681866467,"""0x001138a7ea7c2a5e855400297a52abf22b8b9b5cd71327e861352996018ba3aa""","""Uniswap V3 Pepe/Wrapped Ether 1%""","""0xf239009a101b6b930a527deaab6961b6e7dec8a6""","""WETH""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""PEPE""","""0x6982508145454ce325ddbe47a25d4ec3d2311933"""
1681725263,"""0x0015388d53616b70bdbe609621979ea9344bf1de7b716e1fd76eb73bf755cd48""","""Uniswap V3 Pepe/Wrapped Ether 1%""","""0xf239009a101b6b930a527deaab6961b6e7dec8a6""","""WETH""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""PEPE""","""0x6982508145454ce325ddbe47a25d4ec3d2311933"""


In [8]:
univ3_pools = univ3_df['pool_name'].unique()
univ2_pools = univ2_df['pair_id'].unique()

print(f'univ3 Pepe pools: {univ3_pools}')
print(f'univ2 Pepe pools: {univ2_pools}')

univ3 Pepe pools: shape: (5,)
Series: 'pool_name' [str]
[
	"Uniswap V3 Pepe/McPepes 1%"
	"Uniswap V3 Pepe/Wrapped Ether 1%"
	"Uniswap V3 Pepe/USD Coin 1%"
	"Uniswap V3 McPepes/Wrapped Ether 1%"
	"Uniswap V3 Wojak Coin/Pepe 1%"
]
univ2 Pepe pools: shape: (1,)
Series: 'pair_id' [str]
[
	"0xa43fe16908251ee70ef74718545e4fe6c5ccec9f"
]
